## Parsing Tables

This is a current working example of the parsing and data cleanup for a **specific PDF file** (*https://www.who.int/docs/default-source/coronaviruse/situation-reports/20200121-sitrep-1-2019-ncov.pdf*), after it has been stored on your local file system. The parsing is done using the **PdfMiner Class**. We won't do a full Class overview just yet, we will just look at how it handles, parses and manages the data within this PDF and then how it converts it into a **CSV file**, storing it locally. 


This is the table within this PDF, that we want to extract and parse.

<img style="float: left;" src="https://i.imgur.com/hemEyn1.png">

First, let's import the two main libraries that we will need to accomplish our goals:

In [1]:
import tabula
import pandas as pd

Tabula (https://pypi.org/project/tabula-py/), is the module that we will use to read our PDF tables. You do not need to understand it. You only need to know that it extracts tables from PDF documents, and then converts each one of those tables into a **Pandas DataFrame**.

A **Pandas DataFrame** is a Python object that you will need to understand thoroughly. To put it simply: Pandas DataFrame is nothing but an in-memory representation of an excel sheet via Python programming language. It contains all the methods you will need in order to manipulate your extracted tables. For a lightweight intro, please read this article: https://towardsdatascience.com/pandas-dataframe-a-lightweight-intro-680e3a212b96.

But for now, let's just move on. 

If you scroll down to the bottom of our project file, you will see the class being initialized and its start method being called.

In [ ]:
pdf_parser = PdfMiner(["data/test/20200121-sitrep-1-2019-ncov.pdf"])
pdf_parser.start()

Without going into too much detail, you will see that within the **PdfMiner** class definition, there is a function that deals with the main parsing: **parse_pdf_files**. In its body there is an **IF** clause that calls a single function that will deal with this table parsing specifically. 

That function is **parse_table_example_1**.

In [ ]:
# parses table from Pandas DataFrame object
def parse_table_example_1(self, df):
    """
        Args:
            df (DataFrame): Pandas DataFrame object
        Returns:
            Parsed DataFrame object
    """
    new_columns = []
    for i, column in enumerate(df.columns):
        new_columns.append((column + ' ' +
                            str(df.iloc[0, i]).replace('nan', '')).strip())

    df.columns = new_columns
    df.iloc[8, 0] = ' '.join([df.iloc[8, 0],df.iloc[9, 0]])
    df.drop(index=[0, 9], inplace=True)
    df.reset_index(drop=True, inplace=True)
    df.iloc[1:6, 0] = df.iloc[0, 0]

    return df

Before we head on with function dissection, before we mention the elephant in the room -or rather, Panda in the room- let's process this PDF parsing without special treatment. Just leaving all parsing up to the existing current libraries.

This is our console output for this table. Without special treatment:

<img style="float: left;" src="https://i.imgur.com/CAn7JO0.png">

Okay, what do we need to fix here?
* **Column Labels**
* **Unnecessary/ Incorrect rows & cells**
* **Null values (NaN)**

Let's see how our function handles this.
It starts by accepting a DataFrame object that contains the data representation of our table:

In [ ]:
# parses table from Pandas DataFrame object (df)
def parse_table_example_1(self, df):

Then it will need to create new columns from the available cell data:

In [ ]:
# list where new column labels will se stored
new_columns = []

# loop through each current column label (column in df.columns)
for i, column in enumerate(df.columns):
    # create new column label by adding the current column string and the string from the cell below it
    new_columns.append((column + ' ' +
                        str(df.iloc[0, i]).replace('nan', '')).strip())

Yes, I acknowledge that the final line might be difficult to grasp, right now. 

The **replace** and **strip** methods are Python built-in methods to manipulate strings. We are just replacing 'nan' with an empty string "", then stripping the possible trailing whitespaces.

In [ ]:
df.iloc[0, i]

Pandas has a **df.iloc** method which we can use to select rows and columns by the order in which they appear in the data frame. It takes two arguments where one is to specify rows and the other is to specify columns.

So, here we are just selecting each of the cells below our column labels row **(index=0, column=i)**.

Very briefly, let's do a quick overview of the remaining parsing processes:

In [ ]:
# assign new columns to data frame
df.columns = new_columns

# define cell [8,0] ('Total confirmed') to be cell [8, 0] + cell [9, 0] ('Total confirmed cases')
df.iloc[8, 0] = ' '.join([df.iloc[8, 0],df.iloc[9, 0]])

# drop (delete) two rows (row 0 and row 9)
# inplace parameter - if True, mutates current DataFrame; if False, returns a new copy
df.drop(index=[0, 9], inplace=True)

# reset index (do this everytime you delete a row, even though it wasn't needed here)
# inplace parameter - if True, mutates current DataFrame; if False, returns a new copy
df.reset_index(drop=True, inplace=True)

# set GROUP of null cells in the first column to be the same as cell [0,0] (string 'WPRO')
df.iloc[1:6, 0] = df.iloc[0, 0]

If you're referencing the table picture above, don't be confused by the index values. Remember that we mutated the table several times in most of these lines!

Now, here's the final table, after parsing:

<img style="float: left;" src="https://i.imgur.com/5c8p5ue.png">

Now... that's more like it, right?

If you're worried about the 'nan' value, there's no reason to. It won't be outputted to our final file.

In our main parsing function we then use pandas **df.to_csv** to convert our DataFrame to a CSV file and store it in our local file system.

In [ ]:
df.to_csv(output_path, index=False, encoding='utf-8')

Here is the final CSV file:

<img style="float: left;" src="https://i.imgur.com/nThPDH6.png">